In [ ]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [ ]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [ ]:
# Load Environment Variables
load_dotenv(override=True)

# Gemini Environment Variables
gemini_api_key = env_to_str('GEMINI_API_KEY')
gemini_base_url = env_to_str('GEMINI_BASE_URL')
model_gemini_flash = env_to_str('MODEL_GEMINI_FLASH')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [ ]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [ ]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="gemini_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

In [ ]:
# Gemini Model
gemini_model = OpenAIChatCompletionsModel(
    model=model_gemini_flash,
    openai_client=AsyncOpenAI(
        api_key=gemini_api_key, 
        base_url=gemini_base_url
    )
)

In [ ]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=gemini_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=gemini_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=gemini_model
)

In [ ]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agents):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name):
        if agents:
            for index, agent in enumerate(agents):
                # print(f"  Argument {index+1}: {agent}")
        
                # # Trace Attributes
                # current_span = otel_trace.get_current_span()
                # current_span.set_attribute("user.request", message)
                # current_span.set_attribute("agent.name", agent.name)
                # current_span.set_attribute("model.name", model_gemini_flash)
                results = await asyncio.gather(
                    Runner.run(agent, message)
                )
        else:
            print("No additional arguments provided.")

        outputs = [result.final_output for result in results]
        # result = await Runner.run(agents, message)
        
        # # Add response attributes
        # current_span.set_attribute("response.length", len(result.final_output))
        # current_span.set_attribute("response.preview", result.final_output[:100])
        
        display(Markdown(outputs[0]))
        return outputs
 

In [44]:
result = await phoenix_tracer(
    "Parallel Cold Emails", 
    "Write a cold sales email.", 
    sales_agent1, 
    sales_agent2, 
    sales_agent3
)

Subject: SOC2 Compliance, Simplified with AI

Hi [Name],

Struggling with SOC2? ComplAI automates the process, saving you time and resources. Our AI-powered platform streamlines compliance and prepares you for audits effortlessly.

Interested in a quick demo?

Best,

[Your Name]

ComplAI


Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  Fi